In [54]:
# Indices

import pandas as pd
import numpy as np
from utils.quilt import load_datasets

year = range(2000,2035)

fuel = [     
    "sähkö",
    "lämpö ja kylmä",
    "maakaasu",
    "nestekaasu",
    "lämmitysöljy",
    "diesel",
    "bensiini",
    "ruskohiili",
    "hiili",
    "muut fossiiliset (sis. turve)",
    "kasviöljy",
    "biopolttoaine",
    "muu biomassa",
    "aurinkoterminen",
    "maalämpö",
    "tuulivoima",
    "vesivoima",
    "aurinkosähkö"
]

emission = [
    "hiilidioksidi",
    "metaani",
    "typpidioksidi",
    "typpioksiduuli",
    "hiilimonoksidi",
    "hiukkaset",
    "rikkidioksidi",
    "typenoksidit",
    "fluorihiilivedyt",
    "perfluorihiilivedyt",
    "rikkiheksafluoridi",
    "typpitrifluoridi",
    "F-kaasut",
    "pienhiukkaspäästö",
    "kasvihuonekaasupäästö"
]

sector = [
    "rakennukset",
    "kunnalliset rakennukset, laitteistot/tilat",
    "ei-kunnalliset rakennukset, laitteistot/tilat",
    "asuinrakennukset",
    "julkinen valaistus",
    "teollisuus, ei-ETS",
    "teollisuus, ETS (ei suositella)",
    "kuljetus",
    "kunnalliskalusto",
    "julkinen liikenne",
    "yksityinen ja kaupallinen liikenne",
    "energia",
    "paikallinen sähköntuotanto",
    "paikallinen lämmön ja kylmän tuotanto",
    "vesi",
    "jätteet",
    "maankäytön suunnittelu",
    "maatalous ja metsänhoito (sis. kalanjalostamot)",
    "ympäristö ja biodiversiteetti",
    "terveys",
    "pelastuspalvelut ja hätäpalvelut",
    "turismi",
    "muu sektori",
    "sopeutuminen"
]

subsector_of = [0,0,0,0,0,0,0,7,7,7,7,11,11,11,14,15,16,17,18,19,20,21,22,23]

treatment = ['solid waste disposal', 'composting', 'digestion', 'incineration', 'waste water treatment']

source = ['in - in', 'in - out', 'out - in']

building = ['residential', 'commercial and institutional', 'industry and construction', 'energy industries']

final_use = ['consumer electricity','district heating','gas', 'geothermal', 'heating electricity',
            'industry and machinery','oil heating', 'other heating','road transport']

scopes_of_uses = pd.DataFrame({
    'final_use':final_use,
    'scope':[2,2,2,2,2,1,1,1,1],
    'result':[1]*len(final_use)
}).set_index(['final_use','scope'])

energy_class = ['old', 'new']

mode = ['walking', 'cycling', 'private car', 'bus', 'truck', 'van', 'tram', 'metro', 'train', 'flight', 'boat', 'machinery']

mode0 = ['road','rail','marine','aviation','off-road']

animal = ['cow', 'pig', 'sheep']

landuse = ['forest', 'field', 'turf field', 'peatland']

scope = ['1','2','3']

gas = ['CO2', 'CH4'] # Add PM2.5 and other gases relevant for other impacts

def municipality():
    out = pd.DataFrame({
        'municipality':['Tampere'],
        'result':[1]
    }).set_index(['municipality'])
    return out

# https://ghgprotocol.org/sites/default/files/standards/GHGP_GPC_0.pdf Figure 2
gpc = pd.read_csv("gpc_classification.csv")[['gpc','basic','scope','name']].set_index(['gpc','basic','scope'])

def global_warming_potential():
    df = pd.DataFrame({'gas':gas, 'result':[1,10]})
    return df.set_index('gas')

# A test function to replace groupby for making sure that Iter column is not dropped even if it is not explicitly mentioned.
# This can be used if we end up using probabilistic variables.
# df is the variable dataframe and keep is a list of index column names to keep in the outcome.

def grupby(df, keep):
    out = list(set(df.columns) & set(keep+['Iter']))
    out = df.groupby(out)
    return out

def fillna(object, cols):
    ind = object.index.names
    out = object.reset_index()
    if cols != [None]:
        for i in list(cols):
            a1 = out[out[i].notna()]
            a2 = out[out[i].isna()].drop(i, axis=1).assign(tmp=1)
            addition = pd.DataFrame({i:pd.unique(a1[i]), 'tmp':1})
            a2 = a2.merge(addition).drop('tmp',axis=1)
            out = a1.append(a2)
    if ind != [None]:
        out = out.set_index(ind)
    return out

#display(grupby(ghg_emissions_from_heating_of_stationary_sources().assign(Iter = 1), ['scope']).sum().reset_index())

In [28]:
#tst = load_datasets("jtuomsto/cnh/power_consumption_example")

# datasets from quilt/jyrjola
#
# aluesarjat: Helsingin seudun väestö-, rakennus- ja tulotilastoja 10 tiedostoa
# tst = load_datasets('jyrjola/energiaatlas/buildings') # rakennuskantarekisteri? 77267 rows × 109 columns
# energiateollisuus: kaukolämmön ja sähkön tuotannon tilastoja ja polttoaineen kulutus
#tst = load_datasets('jyrjola/energiateollisuus/electricity_production_hourly') # 78888 rows × 11 columns, 2010-2018
# fingrid: sähkönkulutus ja siirto 2 tiedostoa, esim. 3 min välein
# fingrid_hourly: sähkönkulutus ja hinta, 3 tiedostoa
# fingrid_realtime: 1 tiedosto power, 3/2019
# fmi: 3 tiedostoa: heating_degree_days, solar_radiation_kumpula, solar_radiation savilahti
# hsy: 2 tiedostoa: buildings, pks_khk_paastot
# karttahel: 1 tiedosto: buildings
# lipasto: 3 tiedostoa 
#├─car_unit_emissions # vain henkilöautot
#├─emissions_by_municipality
#└─mileage_per_engine_type
# osm:
#└─helsinki_bike_lane
#statfi
#└─fuel_classification # 84 rows × 10 columns, co2e_emission_factor, calorific_value
#traficom: 11 tiedostoa:
#├─cars_by_engine_type
#├─tf010_kanta_tau_101
#├─tf020_ensirek_tau_102
#├─vehicle_register_2014q4
#├─vehicle_register_2015q4
#├─vehicle_register_2017q3
#├─vehicle_register_2018q1
#├─vehicle_register_2018q2
#├─vehicle_register_2018q3
#├─vehicle_register_2019q1
#└─...
#ymparistotilastot: 19 tiedostoa koskien Helsinkiä/pääkaupunkiseutua?
#├─e03_energian_kokonaiskulutus
#├─e06_ymparistopenniasiakkaat
#├─e12_helsingin_kaukolammon_sahkonkulutus
#├─e13_helen_vantaa_kaukolammon_sahkonhankinta
#├─e14_helen_ominaispaastot
#├─e15_helen_kaukolampo_jaahdytys
#├─e1_sahkonkulutus
#├─e21_kaup_rakennus_tyypeittain
#├─e23_energian_kuluttajaryhmat_kokonaiskulutus
#├─e25_kl_polttoaineet_pks
#└─...

#tst = load_datasets('jyrjola/lipasto/car_unit_emissions')
#display(tst)
#pd.unique(tst['Engine'])

array(['gasoline', 'diesel', 'FFV', 'gas'], dtype=object)

In [128]:
##### Get building data from Statistics Finland

#df = pd.read_csv('http://pxnet2.stat.fi/PXWeb/sq/930e8afc-c473-4288-a656-a6d0478ca5d7')

#df = pd.read_csv('http://pxnet2.stat.fi/PXWeb/pxweb/fi/StatFin/StatFin__asu__rakke/statfin_rakke_pxt_116h.px/table/tableViewLayout2/?loadedQueryId=930e8afc-c473-4288-a656-a6d0478ca5d7&timeType=top&timeValue=1&downloadfile=FileTypeCsvWithHeadingAndSemiColon',
#     sep=";", skiprows=2)

#df = pd.read_json('http://pxnet2.stat.fi/PXWeb/pxweb/fi/StatFin/StatFin__asu__rakke/statfin_rakke_pxt_116h.px/table/tableViewLayout2/?loadedQueryId=930e8afc-c473-4288-a656-a6d0478ca5d7&timeType=top&timeValue=1&downloadfile=FileTypeJsonStat')

#df = pd.read_excel('http://pxnet2.stat.fi/PXWeb/pxweb/fi/StatFin/StatFin__asu__rakke/statfin_rakke_pxt_116h.px/table/tableViewLayout2/?loadedQueryId=930e8afc-c473-4288-a656-a6d0478ca5d7&timeType=top&timeValue=1&downloadfile=FileTypeExcelX',
#                  header=True, sheet_name='006_116h_2019')

import os
import csv
import datetime
import urllib.request
import urllib.parse
import urllib.error
import zlib
import time

"""
from pandas_pcaxis.pxweb_api import PXWebAPI
api = PXWebAPI('http://pxnet2.stat.fi/PXWeb', 'fi')
print(api.list_databases())
print(api.list_topics('StatFin/asu/rakke'))
px_file = api.get_table('StatFin/asu/rakke/statfin_rakke_pxt_116g.px')
df = px_file.to_df()

"""
#import px_reader
#px_obj = px_reader.Px('a_px_file_on_filesystem.px')
#pandas_dataframe = px_obj.pd_dataframe()

def download_px(px_objs, target_dir='.', compressed=False, sleep=1, refresh='check'):
    """
    Fetch PC Axis files for given list of Px objects
    Save the files to target directory

    WARNING: Statfin database contains over 2500 PX files with many gigabytes of data.
    """

    refresh_options = ['never', 'check', 'always']
    if refresh not in refresh_options:
        raise ValueError('Invalid value for refresh, must be one of "{}"'.format(
            '", "'.join(refresh_options)))

    if not isinstance(px_objs, list):
        px_objs = [px_objs]

    for px_obj in px_objs:
        url_parts = urllib.parse.urlparse(px_obj.path)
        # url_parts.path starts with '/'
        target_path = os.path.join(target_dir, url_parts.path[1:])
        target_path = os.path.abspath(target_path)

        if refresh != "always" and os.path.exists(target_path):
            if refresh == 'check':
                if is_latest(px_obj.path, target_path):
                    print('File {} is already latest, skipping'.format(target_path))
                    time.sleep(1)
                    continue
            elif refresh == 'never':
                print('File {} already exists, skipping'.format(target_path))
                continue

        print('Downloading file from {} ...'.format(px_obj.path), end=' ')
        try:
            request = urllib.request.Request(px_obj.path)
            if compressed:
                request.add_header('Accept-encoding', 'gzip')
            response = urllib.request.urlopen(request)
        except urllib.error.HTTPError as e:
            print('ERROR:', e)
            print('Response headers:', e.headers)
            time.sleep(sleep)
            continue

        makedirs(target_path)
        try:
            with open(target_path, 'wb') as f:
                data = response.read()
                if compressed:
                    data = zlib.decompress(data, zlib.MAX_WBITS | 16)
                f.write(data)
        except IOError as e:
            print('ERROR:', e)
            time.sleep(sleep)
            continue

        print('done')
        time.sleep(sleep)

#df = download_px('http://pxnet2.stat.fi/PXWeb/pxweb/fi/StatFin/StatFin__asu__rakke/statfin_rakke_pxt_116h.px/')

#import statsfi_px_api
#import px_reader
#px_info = statsfi_px_api.list_available_px(url="http://pxweb2.stat.fi/database/StatFin/StatFin_rap.csv")

In [152]:
### Data from statfi, manually

#'http://pxnet2.stat.fi/PXWeb/pxweb/fi/StatFin/StatFin__asu__rakke/statfin_rakke_pxt_116h.px/table/tableViewLayout2/?downloadfile=FileTypeCsvWithHeadingAndSemiColon'

df = pd.read_csv('006_116h_2019.csv', skiprows=2, encoding = 'iso8859_10')

#df.columns
"""Index(['Rakennuksen lämmitysaine', 'Alue', 'Vuosi',
       'Rakennuksen kerrosala (m2) Yhteensä',
       'Rakennuksen kerrosala (m2) Erilliset pientalot',
       'Rakennuksen kerrosala (m2) Rivi- ja ketjutalot',
       'Rakennuksen kerrosala (m2) Asuinkerrostalot',
       'Rakennuksen kerrosala (m2) Liikerakennukset',
       'Rakennuksen kerrosala (m2) Toimistorakennukset',
       'Rakennuksen kerrosala (m2) Liikenteen rakennukset',
       'Rakennuksen kerrosala (m2) Hoitoalan rakennukset',
       'Rakennuksen kerrosala (m2) Kokoontumisrakennukset',
       'Rakennuksen kerrosala (m2) Opetusrakennukset',
       'Rakennuksen kerrosala (m2) Teollisuusrakennukset',
       'Rakennuksen kerrosala (m2) Varastorakennukset',
       'Rakennuksen kerrosala (m2) Muut rakennukset'],
      dtype='object')
"""

df.columns = ['final_use','municipality','year','total','detached houses','row house','apartment buildings',
              'commercial buildings','office buildings','transport buildings','health care buildings',
              'congregation buildings','educational buildings','industrial buildings','storage buildings',
              'other buildings'
             ]
df = df.drop('total', axis=1)
df = df[df['final_use'] != 'Yhteensä']
#df = df[df['municipality' != 'KOKO MAA']]

df = pd.melt(df, id_vars=['final_use','municipality','year'], var_name='building', value_name='result')
display(df)

KeyError: True

In [125]:
################ Heating of buildings

## Energy use

def floor_area_of_buildings():
    
    finaluse = [final_use[x] for x in [1,2,3,4,6,7]]
    df = pd.DataFrame({
        'municipality':'Tampere',
        'building':building*len(finaluse)*len(energy_class),
        'final_use':np.repeat(finaluse*len(energy_class), len(building)),
        'energy_class':np.repeat(energy_class,len(building)*len(finaluse)),
        'result':range(len(building)*len(finaluse)*len(energy_class))}) # m²
    return df.set_index(['municipality','building','final_use','energy_class'])

def energy_efficiency_of_buildings():
    df = pd.DataFrame({
        'building':building*len(energy_class),
        'energy_class':np.repeat(energy_class, len(building)),
        'result':[4,5,6,7,1,2,3,4] # kWh/m²/a
    })
    return df.set_index(['building','energy_class'])

def energy_use_from_heating_of_buildings(
    p1 = municipality(),
    p2 = floor_area_of_buildings(),
    p3 = energy_efficiency_of_buildings()):

    out = (p1 * p2).dropna()
    out = out * p3 * 1E-6 # kWh/a --> GWh/a
    
    out = out.groupby(list(set(out.index.names) - {'energy_class'})).sum()
    return out

def emission_factors_of_energy(p1 = scopes_of_uses): # kton/GWh
    out = pd.read_csv('emission_factors_of_energy_consumption.csv')
    out = out.drop(['emission','energy_use','level5'], axis=1).rename(columns={'emission_factor':'result'})
    out = out.set_index(['final_use','year','municipality','emission_trade','gas'])
    out = out * scopes_of_uses
    out = fillna(out, ['municipality'])
    return out

def loss_from_the_grid(): # Loss in the system, given as a ratio to the FINAL energy consumption
    df = pd.DataFrame({
        'final_use':final_use,
        'scope':'3',
        'result':[0.05, 0.04, 0.1, 0.04, 0.02, 0.1, 0.1, 0.1, 0.1]}) # % # ALSO CONTAINS SYSTEM INEFFICIENCIES (hyötysuhde)
    #BUT THIS DEPENDS ON Production type
    return df.set_index(['final_use','scope'])

def emissions_from_heating_of_stationary_sources(
    p1 = emission_factors_of_energy(),
    p2 = energy_use_from_heating_of_buildings(),
    p3 = loss_from_the_grid()):

    out = (p1 * p2).dropna() # kton

    loss = (out.reset_index('scope').drop('scope',1) * p3).dropna()

    out = pd.concat([out.reset_index(), loss.reset_index()])
    out= out.groupby(list(set(out.columns) - {'emission_trade','result'})).sum()

    return out

def ghg_emissions_from_heating_of_stationary_sources(
    p1 = emissions_from_heating_of_stationary_sources(),
    p2 = global_warming_potential()):

    out = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    out = out.groupby(list(set(out.index.names) - {'gas','result','building'})).sum()

    return out

display(emissions_from_heating_of_stationary_sources())

result
municipality final_use        gas  year   scope building                                  
Tampere      district heating CO2e 2005.0 2     commercial and institutional  9.581522e-06
                                                energy industries             2.247302e-05
                                                industry and construction     1.567885e-05
                                                residential                   4.181028e-06
                                          3     commercial and institutional  3.832609e-07
...                                                                                    ...
             other heating    CO2e 2018.0 1     residential                   1.329925e-04
                                          3     commercial and institutional  2.091415e-05
                                                energy industries             3.743096e-05
                                                industry and construction     2.895805e-05
                                                residential                   1.329925e-05

[448 rows x 1 columns]

In [3]:
######### Consumer electricity

def specific_electricity_consumption():
    df = pd.DataFrame({
        'building':building,
        'result':[150, 100, 120, 100] # kWh/m²/a
    })
    return df.set_index('building')

def energy_use_from_consumer_electricity(
    p1 = floor_area_of_buildings(),
    p2 = specific_electricity_consumption()):
    
    p1 = p1.groupby('building').sum()
    p1['final_use'] = 'electricity'
    p1 = p1.set_index('final_use', append = True)
    df = p1 * p2 * 1E-6 # kWh/a --> GWh/a
    df = df.groupby(['building','final_use']).sum()

    return df

def emissions_from_consumer_electricity(
    p1 = energy_use_from_consumer_electricity(),
    p2 = emission_factors_of_energy(),
    p3 = loss_from_the_grid()):

    df = p1 * p2 # ton/a

    loss = df.reset_index('scope').drop('scope',1) * p3

    df = pd.concat([df.reset_index(), loss.reset_index()]).dropna() # non-electricity final uses dropped

    df = df.groupby(['building','final_use','gas','scope']).sum()
    
    return df

def ghg_emissions_from_consumer_electricity(
    p1 = emissions_from_consumer_electricity(),
    p2 = global_warming_potential()):

    df = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    df = df.groupby(['building','final_use','scope']).sum()

    return df

display(ghg_emissions_from_consumer_electricity())

result
building                     final_use   scope          
commercial and institutional electricity 2      0.001159
                                         3      0.000046
energy industries            electricity 2      0.001281
                                         3      0.000051
industry and construction    electricity 2      0.001464
                                         3      0.000059
residential                  electricity 2      0.001647
                                         3      0.000066

In [4]:
# Missing stationary sources:

# I.5.X Agriculture, forestry, and fishing activities
# I.6.X Non-specified sources
# I.7.X Fugitive emissions from mining, processing, storage, and transportation of coal
# I.8.X Fugitive emissions from oil and natural gas systems

In [5]:
################# Transport

def transport_activity_of_people():
    df = pd.DataFrame({
        'transboundary':source,
        'result':[290,50,10] # trips per day
    })
    
    return df.set_index(['transboundary'])

def average_trip_length():
    df = pd.DataFrame({
        'transboundary':source*len(mode),
        'mode':np.repeat(mode, len(source)),
        'result':range(len(mode)*len(source)) # km/trip
    })
    
    return df.set_index(['transboundary','mode'])

def modal_share_of_transport():
    df = pd.DataFrame({
        'transboundary':source*len(mode),
        'mode':np.repeat(mode, len(source)),
        'result':range(len(mode)*len(source)) # fraction
    })
    
    return df.set_index(['transboundary','mode'])

def energy_intensity_of_transport_modes():
    df = pd.DataFrame({
        'mode':mode,
        'fuel':['electricity', 'electricity', 'gasoline', 'diesel', 'diesel', 'diesel',
                'electricity', 'electricity', 'electricity','kerosene','light oil','diesel'],
        'result':range(len(mode)) # MJ/km
    })
    
    return df.set_index(['mode','fuel'])

def emission_factors_of_transport_modes():
    df = pd.DataFrame({
        'fuel':['electricity', 'gasoline', 'diesel','kerosene','light oil']*len(gas),
        'scope':['2','1','1','1','1']*len(gas),
        'gas':np.repeat(gas,5),
        'result':[1, 5, 9, 2, 6, 7, 2, 5, 3,7] # g/MJ = ton/TJ
    })
    
    return df.set_index(['fuel','scope','gas'])

def energy_use_from_transportation( # NOTE! Energy losses are not considered (yet)
    p1 = transport_activity_of_people(),
    p2 = modal_share_of_transport(),
    p3 = energy_intensity_of_transport_modes()):
    
    df = p1 * p2 * p3 * 1E-6 * 365.25 # MJ/d --> TJ/a
    df = df.groupby(['transboundary','fuel','mode']).sum()
    
    return df

def emissions_from_transportation( # NOTE! Energy losses are not considered (yet)
    p1 = energy_use_from_transportation(),
    p2 = emission_factors_of_transport_modes()):
    
    df = p1 * p2 # ton/a
    df = df.groupby(['transboundary','fuel','scope','gas','mode']).sum()
    
    return df

def ghg_emissions_from_transportation(
    p1 = emissions_from_transportation(),
    p2 = global_warming_potential()):
    
    df = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    df = df.groupby(['transboundary','fuel','scope','mode']).sum()
    
    return df

display(ghg_emissions_from_transportation())

result
transboundary fuel        scope mode                 
in - in       diesel      1     bus          0.168735
                                machinery    2.268542
                                truck        0.299973
                                van          0.468707
              electricity 2     cycling      0.022561
                                metro        1.105513
                                train        1.443936
                                tram         0.812214
                                walking      0.000000
              gasoline    1     private car  0.031777
              kerosene    1     flight       0.823653
              light oil   1     boat         2.415033
in - out      diesel      1     bus          0.032325
                                machinery    0.402980
                                truck        0.056029
                                van          0.086199
              electricity 2     cycling      0.005187
                                metro        0.199682
                                train        0.259327
                                tram         0.147817
                                walking      0.000000
              gasoline    1     private car  0.006392
              kerosene    1     flight       0.147269
              light oil   1     boat         0.430264
out - in      diesel      1     bus          0.007111
                                machinery    0.082967
                                truck        0.012068
                                van          0.018317
              electricity 2     cycling      0.001297
                                metro        0.041752
                                train        0.053940
                                tram         0.031119
                                walking      0.000000
              gasoline    1     private car  0.001461
              kerosene    1     flight       0.030506
              light oil   1     boat         0.088829

In [6]:
####### Waste

def amount_of_waste():
    df = pd.DataFrame({
        'treatment':treatment*len(source),
        'source':np.repeat(source, len(treatment)),
        'result':range(len(source)*len(treatment)) # ton/a
    })
    return df.set_index(['treatment','source'])

def emission_factors_of_waste():
    df = pd.DataFrame({
        'treatment':treatment*len(gas),
        'gas':np.repeat(gas, len(treatment)),
        'result':np.repeat([1,2], len(treatment)) # kg/ton
    })
    return df.set_index(['treatment','gas'])

def emissions_from_waste(
    p1 = amount_of_waste(),
    p2 = emission_factors_of_waste()):
    
    df = p1 * p2 * 1E-3 # kg/a --> ton/a
    df = df.groupby(['treatment','gas','source']).sum() # redundant because there are currently no more indices
    
    return df

def ghg_emissions_from_waste(
    p1 = emissions_from_waste(),
    p2 = global_warming_potential()):
 
    df = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    df = df.groupby(['treatment','source']).sum()
    
    return df

display(ghg_emissions_from_waste())

result
treatment             source            
composting            in - in   0.000021
                      in - out  0.000126
                      out - in  0.000231
digestion             in - in   0.000042
                      in - out  0.000147
                      out - in  0.000252
incineration          in - in   0.000063
                      in - out  0.000168
                      out - in  0.000273
solid waste disposal  in - in   0.000000
                      in - out  0.000105
                      out - in  0.000210
waste water treatment in - in   0.000084
                      in - out  0.000189
                      out - in  0.000294

In [7]:
################## AFOLU

def number_of_farm_animals():
    df = pd.DataFrame({
        'animal':animal,
        'result':[10000, 20000, 30000] # number
    })
    
    return df.set_index('animal')

def land_area():
    df = pd.DataFrame({
        'landuse':landuse,
        'result':[40000, 50000, 60000, 70000] # ha
    })
    
    return df.set_index('landuse')

def emission_factors_of_afolu():
    df = pd.DataFrame({
        'source':['livestock']*len(animal)+['land']*len(landuse)+['aggregate']*len(landuse),
        'animal':animal+[None]*len(landuse)*2,
        'landuse':[None]*len(animal)+landuse*2,
        'gas':['CO2']*(len(animal)+len(landuse))+['CH4']*len(landuse),
        'result':[2,3,4,5,6,7,8,9,10,11,12] # g/animal/a or g/ha/a
    })
    return df.set_index(['source','animal','landuse','gas'])

def emissions_from_afolu(
    p1 = number_of_farm_animals(),
    p2 = land_area(),
    p3 = emission_factors_of_afolu()):
    
    df = p1 * p3 * 1E-6 # g/a --> ton/a
    df = df.groupby(['source','gas']).sum()
    
    df2 = p2 * p3 * 1E-6 # g/a --> ton/a
    df2 = df2.groupby(['source','gas']).sum()
    
    df = pd.concat([df, df2])   
    
    return df

def ghg_emissions_from_afolu(
    p1 = emissions_from_afolu(),
    p2 = global_warming_potential()):
    
    df = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    df = df.groupby(['source']).sum()
    
    return df

#display(ghg_emissions_from_afolu())

def why_does_field_appear_in_livestock( ## WTF?!?
    p1 = land_area(),
    p2 = emission_factors_of_afolu()):
    
    df = p1 * p2
    return df

display(emission_factors_of_afolu())
display(land_area())
display(why_does_field_appear_in_livestock())


result
source    animal landuse    gas        
livestock cow    NaN        CO2       2
          pig    NaN        CO2       3
          sheep  NaN        CO2       4
land      NaN    forest     CO2       5
                 field      CO2       6
                 turf field CO2       7
                 peatland   CO2       8
aggregate NaN    forest     CH4       9
                 field      CH4      10
                 turf field CH4      11
                 peatland   CH4      12

,result
landuse,
forest,40000
field,50000
turf field,60000
peatland,70000


result
source    animal landuse    gas        
livestock cow    field      CO2  100000
          pig    field      CO2  150000
          sheep  field      CO2  200000
land      NaN    forest     CO2  200000
                 field      CO2  300000
                 turf field CO2  420000
                 peatland   CO2  560000
aggregate NaN    forest     CH4  360000
                 field      CH4  500000
                 turf field CH4  660000
                 peatland   CH4  840000

In [8]:
############## IPPU

def emissions_from_ippu():
    df = pd.DataFrame({
        'source':['industrial processes','product use']*len(gas),
        'gas':np.repeat(gas,2),
        'result':[100, 20, 5, 8] # ton/a
    })
    
    return df.set_index(['source','gas'])

def ghg_emissions_from_ippu(
    p1 = emissions_from_ippu(),
    p2 = global_warming_potential()):

    df = p1 * p2 * 1E-3 # ton/a --> kton/a (CO2e)
    
    return df

display(ghg_emissions_from_ippu())

,,result
source,gas,
industrial processes,CO2,0.10
product use,CO2,0.02
industrial processes,CH4,0.05
product use,CH4,0.08


In [9]:
##################### Other scope 3

def ghg_emissions_from_other_scope3():
    df = pd.DataFrame({
        'source':['other scope3'],
        'result':[20] # kton/a (CO2e)
    })
    
    return df.set_index(['source'])

display(ghg_emissions_from_other_scope3())

,result
source,
other scope3,20


In [10]:
##### Total GHG emissions from the municipality using GPC classification (compatible with basic/basic+ classification)
    
def total_ghg_emissions_from_the_municipality( # kton/a (CO2e)
    p1 = ghg_emissions_from_heating_of_stationary_sources().reset_index(),
    p2 = ghg_emissions_from_consumer_electricity().reset_index(),
    p3 = ghg_emissions_from_transportation(),
    p4 = ghg_emissions_from_waste().reset_index(),
    p5 = ghg_emissions_from_ippu().reset_index(),
    p6 = ghg_emissions_from_afolu().reset_index(),
    p7 = ghg_emissions_from_other_scope3().reset_index()):
    
    tmp = pd.DataFrame({
        'building':np.repeat(building, 3),
        'scope':['1','2','3']*len(building),
        'gpc':['I.1.1','I.1.2','I.1.3','I.2.1','I.2.2','I.2.3','I.3.1','I.3.2','I.3.3','I.4.1','I.4.2','I.4.3']
    })
    
    tmp = pd.concat([p1, p2]).merge(tmp)
    
    df = tmp.groupby(['gpc']).sum()

    tmp = p3 * pd.DataFrame({ # Half of the emission is allocated to starting point and half to destination
        'transboundary':source,
        'result':[1, 0.5, 0.5]
    }).set_index('transboundary')
    
    tmp = tmp.reset_index().merge(pd.DataFrame({
        'mode':mode,
        'mode0':['road']*6 + ['rail']*3 + ['marine'] + ['aviation'] + ['off-road']
    })).merge(pd.DataFrame({
        'scope':scope*len(mode0),
        'mode0':np.repeat(mode0, len(scope)),
        'gpc':['II.1.1','II.1.2','II.1.3','II.2.1','II.2.2','II.2.3','II.3.1','II.3.2','II.3.3','II.4.1','II.4.2','II.4.3','II.5.1','II.5.2','II.5.3']
    }))

    tmp = tmp.groupby(['gpc']).sum()#['result'] # kton/a (CO2e)

    df = pd.concat([df, tmp])

    tmp = p4.merge(pd.DataFrame({
        'treatment':np.repeat(treatment, len(source)),
        'source':source*len(treatment),
        'gpc':['III.1.1','III.1.2','III.1.3','III.2.1','III.2.2','III.2.3','III.2.1','III.2.2','III.2.3','III.3.1','III.3.2','III.3.3','III.4.1','III.4.2','III.4.3']
    }))
    tmp = tmp.groupby('gpc').sum()

    df = pd.concat([df, tmp])

    tmp = p5.merge(pd.DataFrame({
        'source':['industrial processes','product use'],
        'gpc':['IV.1','IV.2']
    }))
    tmp = tmp.groupby('gpc').sum()

    df = pd.concat([df, tmp])

    tmp = p6.merge(pd.DataFrame({
        'source':['livestock','land','aggregate'],
        'gpc':['V.1','V.2','V.3']
    }))
    tmp = tmp.groupby('gpc').sum()

    df = pd.concat([df, tmp])

    tmp = p7.merge(pd.DataFrame({
        'source':['other scope3'],
        'gpc':['VI.1']
    }))
    tmp = tmp.groupby('gpc').sum()

    df = pd.concat([df, tmp])
    df = df.reset_index().merge(gpc.reset_index()).set_index(['name','gpc','basic','scope'])

    return df

display(total_ghg_emissions_from_the_municipality())

,,,,result
name,gpc,basic,scope,
residential fuel,I.1.1,basic,1,0.000015
residential grid,I.1.2,basic,2,0.001655
residential losses,I.1.3,basic+,3,0.000067
commercial fuel,I.2.1,basic,1,0.000025
commercial grid,I.2.2,basic,2,0.001174
commercial losses,I.2.3,basic+,3,0.000048
manufacturing fuel,I.3.1,basic,1,0.000035
manufacturing grid,I.3.2,basic,2,0.001487
manufacturing losses,I.3.3,basic+,3,0.000061


In [11]:
####### Total energy use and GHG emissions using SECAP reporting

### NOTE! The end result should be more specific to be useful for SECAP

def energy_use_of_the_municipality( # GWh/a
    p1 = energy_use_from_heating_of_buildings().reset_index(),
    p2 = energy_use_from_consumer_electricity().reset_index(),
    p3 = energy_use_from_transportation().reset_index()):
    
    p1['sector'] = 'heating'
    p2['sector'] = 'electricity'
    p3['sector'] = 'transportation'
    
    df = pd.concat([p1, p2])
    p3['final_use'] = p3['fuel']
    p3['result'] / 3.6 # TJ/a --> GWh/a
    df = pd.concat([df[['sector','final_use','result']], p3[['sector','final_use','result']]])
    df = df.groupby(['sector','final_use']).sum()
    
    return df

display(energy_use_of_the_municipality())

result
sector         final_use             
electricity    electricity   0.091000
heating        district      0.000258
               electricity   0.000386
               gas           0.000514
               geothermal    0.000642
               oil           0.000770
transportation diesel       66.168690
               electricity  58.089360
               gasoline      1.585185
               kerosene     31.294620
               light oil    38.606925

In [18]:
#class MyClass(object): pass

#objs = [MyClass() for i in range(50)]

#display(objs)

class MyClass(object):
    def __init__(self, number):
        self.number = number

my_objects = []

for i in range(100):
    my_objects.append(MyClass(i))

# later

for obj in my_objects:
    print(obj)